[View in Colaboratory](https://colab.research.google.com/github/plushvoxel/Project-Lernende-Agenten-colab/blob/master/Code_with_automation_testing_and_logging_not_finished.ipynb)

In [1]:
from __future__ import print_function

import math
from urllib import request
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from google.colab import files
from tarfile import open as taropen
from struct import unpack
import os
import glob
import math
import seaborn as sns
import csv
import time
import base64

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

request.urlretrieve("https://github.com/plushvoxel/Project-Lernende-Agenten-Data-Generator/blob/master/frequency.tar?raw=true", "frequency.tar")
tar = taropen("frequency.tar")

data = dict()
MODKEY = "mod"

for member in tar.getmembers():
  modulation = member.name.split('_')[0]
  if modulation == "am":
    modulation = 0
  else:
    modulation = 1
  if not MODKEY in data:
    data[MODKEY] = [modulation]
  else:
    data[MODKEY].append(modulation)
  with tar.extractfile(member) as f:
    buffer = f.read()
    num_floats = len(buffer)//4
    floats = unpack("f"*num_floats, buffer)
    i = floats[0::2]
    q = floats[1::2]
    for j in range(len(i)):
      ikey = "i{:05d}".format(j)
      qkey = "q{:05d}".format(j)
      if not ikey in data:
        data[ikey] = [i[j]]
      else:
        data[ikey].append(i[j])
      if not qkey in data:
        data[qkey] = [q[j]]
      else:
        data[qkey].append(q[j])        
signal_dataframe = pd.DataFrame(data=data)
signal_dataframeReal = signal_dataframe.copy()
signal_dataframe = signal_dataframe.reindex(np.random.permutation(signal_dataframe.index))
print(signal_dataframe)



      i00000  i00001  i00002  i00003  i00004  i00005  i00006  i00007  i00008  \
1678     0.9    -0.5    -0.4     1.0    -0.7    -0.2     0.6    -0.4    -0.0   
1789    -0.1     0.2    -0.5     0.5    -0.2    -0.1     0.3    -0.7     1.0   
298     -1.3    29.7  -158.9   325.1  -303.9   127.9   -19.2    -0.9    19.0   
1192    -0.3     0.3    -0.2     0.2    -0.4     0.6    -0.6     0.1     0.2   
627   -143.3    69.3    93.8  -207.6   160.6   -24.7   -61.2    63.7   -32.8   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
1119    -0.2     0.1    -0.2     0.1    -0.1    -0.1     0.1    -0.1    -0.0   
1379    -0.2     0.1     0.0    -0.0     0.0    -0.1     0.2    -0.2     0.2   
415     36.1   -90.9   189.1  -180.8    78.0    -9.5   -13.3    25.9   -22.8   
498    -17.8   284.4  -441.0   313.4  -118.3   102.1  -153.7   117.4   -49.0   
139    385.4  -261.6    75.6    -6.2    -0.0    -0.0     0.1    -0.1     0.1   

      i00009   ...    q02038  q02039  q

In [0]:
def parse_labels_and_features(dataset):
  """Extracts labels and features.
  
  This is a good place to scale or transform the features if needed.
  
  Args:
    dataset: A Pandas `Dataframe`, containing the label on the first column and
      monochrome pixel values on the remaining columns, in row major order.
  Returns:
    A `tuple` `(labels, features)`:
      labels: A Pandas `Series`.
      features: A Pandas `DataFrame`.
  """
  labels = dataset[MODKEY]

  # DataFrame.loc index ranges are inclusive at both ends.
  features = dataset.iloc[:,1:4097]
  return labels, features

In [0]:
def construct_feature_columns():
  """Construct the TensorFlow Feature Columns.

  Returns:
    A set of feature columns
  """ 
  
  # There are 784 pixels in each image.
  return set([tf.feature_column.numeric_column('features', shape=4096)])

In [0]:
def create_predict_input_fn(features, labels, batch_size, repeat_count = 1):
  """A custom input_fn for sending mnist data to the estimator for predictions.

  Args:
    features: The features to base predictions on.
    labels: The labels of the prediction examples.

  Returns:
    A function that returns features and labels for predictions.
  """
  def _input_fn():
    raw_features = {"features": features.values}
    raw_targets = np.array(labels)
    
    ds = Dataset.from_tensor_slices((raw_features, raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(repeat_count)
    
        
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
def create_training_input_fn(features, labels, batch_size, num_epochs=None, shuffle=True, repeat_count=1):
  """A custom input_fn for sending MNIST data to the estimator for training.

  Args:
    features: The training features.
    labels: The training labels.
    batch_size: Batch size to use during training.

  Returns:
    A function that returns batches of training features and labels during
    training.
  """
  def _input_fn(num_epochs=None, shuffle=True):
    # Input pipelines are reset with each call to .train(). To ensure model
    # gets a good sampling of data, even when number of steps is small, we 
    # shuffle all the data before creating the Dataset object
    idx = np.random.permutation(features.index)
    raw_features = {"features":features.reindex(idx)}
    raw_targets = np.array(labels[idx])
   
    ds = Dataset.from_tensor_slices((raw_features,raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(repeat_count)
    
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
def train_nn_classification_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets,
    basepath,
    filewriter):
  """Trains a neural network classification model for the MNIST digits dataset.
  
  In addition to training, this function also prints training progress information,
  a plot of the training and validation loss over time, as well as a confusion
  matrix.
  
  Args:
    learning_rate: An `int`, the learning rate to use.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing the training features.
    training_targets: A `DataFrame` containing the training labels.
    validation_examples: A `DataFrame` containing the validation features.
    validation_targets: A `DataFrame` containing the validation labels.
      
  Returns:
    The trained `DNNClassifier` object.
  """

  periods = 10
  # Caution: input pipelines are reset with each call to train. 
  # If the number of steps is small, your model may never see most of the data.  
  # So with multiple `.train` calls like this you may want to control the length 
  # of training with num_epochs passed to the input_fn. Or, you can do a really-big shuffle, 
  # or since it's in-memory data, shuffle all the data in the `input_fn`.
  steps_per_period = steps / periods  
  # Create the input functions.
  predict_training_input_fn = create_predict_input_fn(
    training_examples, training_targets, batch_size)
  predict_validation_input_fn = create_predict_input_fn(
    validation_examples, validation_targets, batch_size)
  training_input_fn = create_training_input_fn(
    training_examples, training_targets, batch_size)
  
  # Create the input functions.
  predict_training_input_fn = create_predict_input_fn(
    training_examples, training_targets, batch_size)
  predict_validation_input_fn = create_predict_input_fn(
    validation_examples, validation_targets, batch_size)
  training_input_fn = create_training_input_fn(
    training_examples, training_targets, batch_size)
  
  # Create feature columns.
  feature_columns = [tf.feature_column.numeric_column('features', shape=4096)]

  # Create a DNNClassifier object.
  my_optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  classifier = tf.estimator.DNNClassifier(
      feature_columns=feature_columns,
      n_classes=2,
      hidden_units=hidden_units,
      optimizer=my_optimizer,
      config=tf.contrib.learn.RunConfig(keep_checkpoint_max=1)
  )

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("LogLoss error (on validation data):")
  training_errors = []
  validation_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    classifier.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
  
    # Take a break and compute probabilities.
    training_predictions = list(classifier.predict(input_fn=predict_training_input_fn))
    training_probabilities = np.array([item['probabilities'] for item in training_predictions])
    training_pred_class_id = np.array([item['class_ids'][0] for item in training_predictions])
    training_pred_one_hot = tf.keras.utils.to_categorical(training_pred_class_id,2)
        
    validation_predictions = list(classifier.predict(input_fn=predict_validation_input_fn))
    validation_probabilities = np.array([item['probabilities'] for item in validation_predictions])    
    validation_pred_class_id = np.array([item['class_ids'][0] for item in validation_predictions])
    validation_pred_one_hot = tf.keras.utils.to_categorical(validation_pred_class_id,2)    
    
    # Compute training and validation errors.
    training_log_loss = metrics.log_loss(training_targets, training_pred_one_hot)
    validation_log_loss = metrics.log_loss(validation_targets, validation_pred_one_hot)
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, validation_log_loss))
    # Add the loss metrics from this period to our list.
    training_errors.append(training_log_loss)
    validation_errors.append(validation_log_loss)
  print("Model training finished.")
  # Remove event files to save disk space.
  _ = map(os.remove, glob.glob(os.path.join(classifier.model_dir, 'events.out.tfevents*')))
  
  # Calculate final predictions (not probabilities, as above).
  final_predictions = classifier.predict(input_fn=predict_validation_input_fn)
  final_predictions = np.array([item['class_ids'][0] for item in final_predictions])
  
  
  accuracy = metrics.accuracy_score(validation_targets, final_predictions)
  print("Final accuracy (on validation data): %0.2f" % accuracy)
  # Output a graph of loss metrics over periods.
  plt.ylabel("LogLoss")
  plt.xlabel("Periods")
  plt.title("LogLoss vs. Periods")
  plt.plot(training_errors, label="training")
  plt.plot(validation_errors, label="validation")
  plt.legend()

  firstPic = base64.b64encode(plt.show())
  
  # Output a plot of the confusion matrix.
  cm = metrics.confusion_matrix(validation_targets, final_predictions)
  # Normalize the confusion matrix by row (i.e by the number of samples
  # in each class).
  cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
  ax = sns.heatmap(cm_normalized, cmap="bone_r")
  ax.set_aspect(1)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")
  plt.show()
  #['Model', 'learning rate', 'training set size', 'validating set size', 'accuracy']
  secondPic = base64.b64encode(plt.show())
  filewriter.writerow([hidden_units, learning_rage, training_examples.shape[0],validating_examples.shape[0] , accuracy, firstPic, secondPic])
  return classifier

In [0]:
def train_automated(
    training_set_size,
    validating_set_size,
    test_set_size,
    learning_rate,
    steps,
    batch_size,
    model,
    basepath,
    filewriter):
  """ Function used for automate the process of trying new network configurations
  
  Args:
    training_set_size: An 'int', number of samples used for training
    validating_set_size: An 'int', number of samples used for validation
    test_set_size: An 'int', number of samples used for test
    learning_rate: An `int`, the learning rate to use.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    model: A list of 'int', define the number of neurons in each hidden layer
      
  Returns:
    The trained `DNNClassifier` object.
  """
  activation_function = "RELU" #@param ["RELU", "Sigmoid", "Tanh"]
  regression = "None" #@param ["None", "L1", "L2"]
  regression_rate = 3 #@param ["3", "1", "0.3", "0.1", "0.03", "0.01", "0.003", "0.001"] {type:"raw"}
  filepath = ""
  training_targets, training_examples = parse_labels_and_features(signal_dataframe[0:training_set_size])
  validation_targets, validation_examples = parse_labels_and_features(signal_dataframe[training_set_size:(training_set_size+validating_set_size)])
  test_targets, test_examples = parse_labels_and_features(signal_dataframe[(training_set_size+validating_set_size):(training_set_size+validating_set_size+test_set_size)])
  nn_classification = train_nn_classification_model(
      learning_rate=learning_rate,
      steps=steps,
      batch_size=batch_size,
      hidden_units=model,
      training_examples=training_examples,
      training_targets=training_targets,
      validation_examples=validation_examples,
      validation_targets=validation_targets,
      basepath  = filepath,
      filewriter = filewriter)

In [20]:
num_samples = signal_dataframe.shape[0]
learning_rate_steps = [0.001, 0.003, 0.01, 0.03]
data_set_distribution= [[60, 20, 20]]
basepath = "C:\\Users\\Ahmad\\Documents\\Studium\\WS1819\\LA\\" + time.strftime("%Y%m%d-%H%M%S")+".csv"
with open(basepath, 'w') as csvfile:
  filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
  filewriter.writerow(['Model', 'learning rate', 'training set size', 'validating set size', 'accuracy'])
  for x in range(1,4): # number of layers
    for z in range(500,3000,500): #number of neurons per layer
      for y in learning_rate_steps: # learning_rate used for training
        for v in data_set_distribution: # try several dataset_distributions
          training_set_size = int(num_samples * (v[0]/100))
          validating_set_size = int(num_samples * (v[1]/100))
          test_set_size = int(num_samples * (v[2]/100))
          model = [0] * x
          for a in range(x):
            if a == 0:
              model[a] = z
            else:
              model[a] = model[a-1]/2

          print(training_set_size)
          print(validating_set_size)
          print(test_set_size)
          print(y)
          print(model)
          train_automated(
          training_set_size,
          validating_set_size,
          test_set_size,
          y,
          153,
          10,
          model,
          basepath,
          filewriter)

        
      
    

1224
408
408
0.001
[500]
Training model...
LogLoss error (on validation data):
  period 00 : 6.94
  period 01 : 4.74
  period 02 : 3.81
  period 03 : 3.13


KeyboardInterrupt: ignored

In [0]:
name = 'test.csv'
with open(name, 'w') as csvfile:
  filewriter = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
  filewriter.writerow(['Model', 'learning rate', 'training set size', 'validating set size', 'accuracy'])
  files.download('test.csv')